In [1]:
import numpy as np 
import pandas as pd 
from tqdm import tqdm 
import datetime as dt 
import matplotlib.pyplot as plt


########################################################### 
# Updated test code
data = pd.read_csv('data/stock_returns.csv', parse_dates=['Date'], index_col='Date').dropna(axis=1)
data = data["2000-01-01":].dropna(axis = 1)
target = np.random.choice(data.columns, 1)[0]
target = "SPY"
features = data.drop(columns=target).iloc[:, :]
x = features.copy()
y = data[target].copy()

print(y.name)

SPY


In [2]:
import sys 
sys.path.append('../')
from main import Analyze

# Setup Config: 

# Example configuration
stationarity_cfg = {
    'adf': {'max_diff': 5, 'significance': 0.05},
    'kpss': {'significance': 0.05},
    'pp': {'significance': 0.05},
    'structural_break': True,
    'gls': False,
    'nonlinear': True
}

causality_cfg = {
    'significance_level': 0.05,
    'max_lag': 4
}

In [3]:
ac = Analyze(verbose = False, stationarity_config=stationarity_cfg, causality_config=causality_cfg)

In [4]:
res = ac.results(x, y, decompose = False)

2025-02-06 19:06:57,556 - src.causality_logic - INFO - CausalityAnalyzer initialized with significance_level=0.05, max_lag=4
Stationarity SPY: 100%|██████████| 48/48 [08:33<00:00, 10.70s/it] 
2025-02-06 19:16:04,065 - src.causality_logic - INFO - Starting unified causality tests for target: SPY
2025-02-06 19:16:04,066 - src.causality_logic - INFO - Starting Granger causality tests for target: SPY
Granger Causality: SPY -> ORCL: 100%|██████████| 47/47 [00:01<00:00, 45.66it/s]
2025-02-06 19:16:05,099 - src.causality_logic - INFO - Completed Granger causality tests for target: SPY
2025-02-06 19:16:05,129 - src.causality_logic - INFO - Starting instantaneous causality test for target: SPY
2025-02-06 19:16:07,468 - src.causality_logic - WARNING - PEP Perfect Instaneous Causality detected for SPY.
2025-02-06 19:16:08,616 - src.causality_logic - WARNING - MMM Perfect Instaneous Causality detected for SPY.
2025-02-06 19:16:09,772 - src.causality_logic - WARNING - WMT Perfect Instaneous Causali

In [5]:
# Standard Ouptut of the results
res.keys()

dict_keys(['stationarity_report', 'var_model', 'causality', 'new_data'])

In [7]:
m = res['causality']['impulse_response']

In [37]:
# m.plot(impulse='SPY', response='PC9', figsize=(5, 4))
# plt.show()

In [38]:
res['var_model'].fevd().summary()

FEVD for PFE
          PFE       PEP       MMM       WMT       CVX        MO      INTC        MU       RRC       WFC       MRK      QCOM       MOS       TGT         F       IBM       AMD       XOM      SBUX        KR       RTX       WBA        BA        GE        AA       DIS      CSCO       CVS        KO      CTRA       TSN       OXY         C         X       EOG       CLX       JPM         K      SONY       CAG       PGR       AFL       AIG      AMAT      DLTR      ADBE      ORCL       SPY
0    1.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000
1    0.987905  0.00

In [14]:
colnames = res['new_data'].columns.to_list()
impulse_response_lag_1 = pd.DataFrame(m.irfs[1, :, :], columns=colnames, index = colnames)
# impulse_response_lag_1['SPY'].sort_values(ascending=False)
impulse_response_lag_1.style.background_gradient(cmap='coolwarm')

,PFE,PEP,MMM,WMT,CVX,MO,INTC,MU,RRC,WFC,MRK,QCOM,MOS,TGT,F,IBM,AMD,XOM,SBUX,KR,RTX,WBA,BA,GE,AA,DIS,CSCO,CVS,KO,CTRA,TSN,OXY,C,X,EOG,CLX,JPM,K,SONY,CAG,PGR,AFL,AIG,AMAT,DLTR,ADBE,ORCL,SPY
PFE,-0.011872,-0.007892,-0.023943,-0.039458,-0.024088,-0.009740,-0.001929,0.028477,-0.068189,-0.026557,-0.005817,-0.006561,-0.010969,0.009180,0.013049,-0.018944,0.021204,0.008387,-0.065537,0.064323,-0.000140,-0.003306,0.031918,0.026366,-0.012414,-0.025232,-0.014734,-0.074151,-0.042645,0.019892,-0.002664,0.030284,-0.092028,0.001524,-0.027488,-0.076578,-0.043759,0.007336,-0.021773,0.039850,-0.014054,0.066898,0.159068,-0.040834,0.027045,-0.003459,0.047681,0.000756
PEP,0.011018,-0.125445,-0.057694,0.004951,-0.036275,0.002318,-0.019352,0.000959,-0.023868,-0.009161,-0.023197,-0.002729,0.001069,0.013239,0.005838,0.002008,-0.004517,0.012366,-0.018733,0.028450,-0.012503,-0.007682,0.033419,0.021675,-0.001053,-0.031123,-0.001181,0.004410,0.005766,0.013995,-0.020214,0.021545,-0.060662,-0.004694,-0.006195,-0.009290,-0.026891,0.006910,-0.007259,0.053311,-0.036420,0.032689,0.009952,-0.002086,-0.008383,-0.042137,0.010105,0.000412
MMM,0.008034,0.008069,-0.021097,0.008780,-0.026277,-0.006128,-0.003957,-0.001007,-0.020690,-0.009059,-0.044476,-0.018695,0.005538,0.000360,-0.002690,0.000478,0.008064,0.029835,-0.004736,-0.012297,0.049952,-0.008183,0.021657,0.005765,0.022871,-0.046039,-0.012368,0.011749,-0.000882,0.013300,-0.000435,-0.028498,-0.017834,-0.001419,0.008992,0.018560,-0.005679,0.025204,-0.001006,-0.034080,-0.029935,0.003136,0.001481,0.008520,0.015069,0.000143,0.008179,0.000043
WMT,0.007628,-0.080974,-0.053289,-0.024591,-0.057162,-0.006085,-0.022712,0.013519,0.002856,0.013483,-0.035184,-0.000628,-0.015424,-0.008210,0.006919,-0.024999,-0.018260,0.025848,-0.046020,0.035490,-0.031740,-0.000099,0.001991,-0.005767,-0.016996,-0.057698,0.008351,-0.027944,-0.002017,0.006966,-0.052143,0.002260,-0.030891,0.005374,-0.042533,-0.014168,-0.032168,0.016832,-0.000936,0.065471,0.029478,-0.032797,0.006226,0.025613,0.019597,-0.001875,-0.007447,0.001870
CVX,-0.002852,-0.057902,0.002540,-0.014159,-0.053342,-0.002313,0.000642,0.011493,-0.020168,-0.047698,-0.050608,-0.033722,0.001347,0.017726,0.017253,-0.020464,0.000930,0.010642,0.042982,0.008417,0.033420,-0.002891,-0.010370,0.015936,0.041644,-0.005694,-0.010080,-0.001370,0.027120,0.022146,0.071848,0.000482,-0.014395,-0.002642,0.022859,-0.063907,-0.036907,-0.010476,-0.019429,-0.009067,0.032068,0.031161,0.025884,0.003344,-0.005113,0.019769,0.007243,-0.000040
MO,0.023090,-0.017242,-0.042103,-0.034133,-0.016907,-0.012037,-0.017848,-0.023230,-0.036058,0.022667,-0.038967,-0.014842,0.003390,0.023986,0.006384,0.012967,-0.002680,-0.012091,0.026698,-0.020198,0.030007,-0.012646,0.031446,0.029198,-0.016250,-0.025674,-0.036117,-0.005125,0.025242,0.057578,0.007351,0.024785,0.037469,-0.018524,-0.040400,0.023991,-0.056403,0.000577,0.006009,-0.016890,0.002087,0.011626,-0.017973,0.001734,-0.064043,-0.020291,-0.011258,0.001170
INTC,0.031496,-0.165968,0.035917,-0.025183,-0.018890,0.009356,-0.034796,0.103166,0.043758,-0.010552,-0.127103,-0.073999,0.006155,0.037816,-0.000427,0.050737,0.007119,0.051670,0.002286,0.001643,0.000206,-0.008404,0.014300,0.036764,0.059834,-0.034625,-0.122818,0.014835,-0.068110,-0.110834,0.022135,-0.014941,-0.276357,-0.003411,-0.003313,0.057240,0.021832,-0.028685,0.075159,0.056387,0.021724,0.016992,-0.008201,0.037069,0.027877,-0.104543,0.011549,-0.001388
MU,0.002299,-0.100306,0.003465,-0.008689,-0.030150,-0.000109,0.008171,-0.008981,0.052055,-0.013572,-0.029784,-0.041049,-0.016294,-0.007990,0.000379,-0.002114,0.013910,0.025925,0.019396,-0.020446,-0.002076,0.010682,0.005245,0.032566,0.014013,-0.059544,-0.052605,-0.019802,0.055093,-0.021324,0.010752,-0.011569,-0.078830,-0.003086,-0.011890,-0.021515,0.031118,-0.001363,0.034258,0.011839,0.093011,-0.044038,0.007048,0.016445,0.025216,-0.082918,0.031212,0.000427
RRC,-0.009341,-0.012864,-0.003542,0.007250,-0.013504,-0.008217,0.000433,0.001627,-0.033369,-0.043895,0.007129,0.000584,-

In [10]:
m.G[-1].shape

(2304, 2304)

In [11]:
cum_cov_effect = m.cum_effect_cov()

In [12]:
cum_cov_effect.shape

(11, 2304, 2304)

In [21]:
len(colnames) ** 2

2304